In [1]:
import pandas as pd
import os
import re
import numpy as np
pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
results_folder_kdd = "../results/kdd/"
results_folder_prsa = "../results/prsa/"

In [3]:
def find_results_txt(root_dir, data=None):
    results = []
    for root, dirs, files in os.walk(root_dir):

        if ("result.txt" in files) and ("run0log.txt" in files):
            for file in files:
                if file == "result.txt":
                    path_components = root.split(os.sep)
                    with open(os.path.join(root, file), 'r') as f:
                        text = f.read()
                        if data == "prsa":
                            pattern = r"Test score \(RMSE, MAPE\) for prsa: \((\d+\.\d+), (\d+\.\d+)\)"
                        elif data == "kdd":
                            pattern = r"Test score \(F1, AP\) for kdd: \((\d+\.\d+), (\d+\.\d+)\)"
                        match = re.search(pattern, text)
                        if match:
                            rmse = round(float(match.group(1)), 2)
                            mape = round(float(match.group(2)), 2)
                        else:
                            rmse = 0
                            mape = 0
                if file == "run0log.txt":
                    path_components = root.split(os.sep)
                    with open(os.path.join(root, file), 'r') as f:
                        text = f.read()
                        pattern = r"model params: (\d+(,\d+)*)"
                        match = re.search(pattern, text)
                        if match:
                            nparams = float(match.group(1).replace(",", ""))
                        else:
                            nparams = 0
                        
            results.append([root, path_components[-1], rmse, mape, nparams])
            
    return results

def name_prsa(row):
    config = ""
    if (
        (row["fieldtransflayers"] == 1) or (row["fieldtransflayers"] == 2) 
    ):
        config += "Final Transf." 
    if (
        not ((row["fieldtransflayers"] == 1) or (row["fieldtransflayers"] == 2))
    ):
        config += "Field Transf." 

    return config

def name_kdd(row):
    config = ""
    if (
        (row["fieldtransflayers"] == 1) or (row["fieldtransflayers"] == 2) 
    ):
        config += "Final Transf." 
    if (
        not ((row["fieldtransflayers"] == 1) or (row["fieldtransflayers"] == 2))
    ):
        config += "Field Transf." 
   
    return config

def extract_pos_emb(text):
    if "_posemb_" in text:
        return "Yes"
    return "No"

def extract_col_emb(text):
    if "_colemb_" in text:
        return "Yes"
    return "No"

def create_dataframe(results, data=None):
    columns = ["root", "path", "RMSE", "MAPE", "# weights"]
    df = pd.DataFrame(results, columns=columns)

    df["Model"] = df['path'].str.split("_", expand=False).str[0]
    df["nlayers"] = df['path'].str.extract(r'(\d+)layers_', expand=False).astype(float)
    df["fieldtransflayers"] = df['path'].str.extract(r'(\d+(?=fieldtransflayers_|fieldtranfslayers_))', expand=False).astype(float)
    df["hs"] = df['path'].str.extract(r'(\d+)hs_', expand=False).astype(float)
    df["pt_ep"] = df['path'].str.extract(r'pt(\d+)ep_', expand=False).astype(float)
    df["ft_ep"] = df['path'].str.extract(r'ft(\d+)ep_', expand=False).astype(float)
    df["seed"] = df['path'].str.extract(r'_seed(\d+)', expand=False).astype(float)
    df["Pos. emb."] = df["path"].map(extract_pos_emb)
    df["Col. emb."] = df["path"].map(extract_col_emb)

    # Group characteristics into generic names
    if data == "prsa":
        df["MoreCapacity"] = df.apply(lambda row: name_prsa(row), axis=1)
    if data == "kdd":
        df["MoreCapacity"] = df.apply(lambda row: name_kdd(row), axis=1)
 
    # Sort
    df = df.sort_values(by=["Model", "RMSE"], ascending=[True, False])

    # Rename if KDD
    if data == "kdd":
        df = df.rename(columns={'RMSE': 'F1', 'MAPE': 'AP'})

    return df

results_kdd = find_results_txt(results_folder_kdd, data="kdd")
df_kdd = create_dataframe(results_kdd, data="kdd")

results_prsa = find_results_txt(results_folder_prsa, data="prsa")
df_prsa = create_dataframe(results_prsa, data="prsa")

In [4]:
df_kdd.groupby(["Model", "MoreCapacity"]).count()

root  path  F1  AP  # weights  nlayers  \
Model         MoreCapacity                                            
ColumnTabBert Field Transf.    20    20  20  20         20       20   
              Final Transf.    20    20  20  20         20       20   
FTTransformer Field Transf.    20    20  20  20         20       20   
Fieldy        Field Transf.    20    20  20  20         20       20   
              Final Transf.    20    20  20  20         20       20   
RowTabBert    Field Transf.    20    20  20  20         20       20   
              Final Transf.    20    20  20  20         20       20   
Tabbie        Field Transf.    20    20  20  20         20       20   

                             fieldtransflayers  hs  pt_ep  ft_ep  seed  \
Model         MoreCapacity                                               
ColumnTabBert Field Transf.                 20  20     20     20    20   
              Final Transf.                 20  20     20     20    20   
FTTransformer Field Transf.                 20  20     20     20    20   
Fieldy        Field Transf.                 20  20     20     20    20   
              Final Transf.                 20  20     20     20    20   
RowTabBert    Field Transf.                 20  20     20     20    20   
              Final Transf.                 20  20     20     20    20   
Tabbie        Field Transf.                 20  20     20     20    20   

                             Pos. emb.  Col. emb.  
Model         MoreCapacity                         
ColumnTabBert Field Transf.         20         20  
              Final Transf.         20         20  
FTTransformer Field Transf.         20         20  
Fieldy        Field Transf.         20         20  
              Final Transf.         20         20  
RowTabBert    Field Transf.         20         20  
              Final Transf.         20         20  
Tabbie        Field Transf.         20         20

In [5]:
df_prsa.groupby(["Model", "MoreCapacity"]).count()

root  path  RMSE  MAPE  # weights  nlayers  \
Model         MoreCapacity                                                
ColumnTabBert Field Transf.    20    20    20    20         20       20   
              Final Transf.    20    20    20    20         20       20   
FTTransformer Field Transf.    20    20    20    20         20       20   
Fieldy        Field Transf.    20    20    20    20         20       20   
              Final Transf.    20    20    20    20         20       20   
RowTabBert    Field Transf.    20    20    20    20         20       20   
              Final Transf.    20    20    20    20         20       20   
Tabbie        Field Transf.    20    20    20    20         20       20   

                             fieldtransflayers  hs  pt_ep  ft_ep  seed  \
Model         MoreCapacity                                               
ColumnTabBert Field Transf.                 20  20     20     20    20   
              Final Transf.                 20  20     20     20    20   
FTTransformer Field Transf.                 20  20     20     20    20   
Fieldy        Field Transf.                 20  20     20     20    20   
              Final Transf.                 20  20     20     20    20   
RowTabBert    Field Transf.                 20  20     20     20    20   
              Final Transf.                 20  20     20     20    20   
Tabbie        Field Transf.                 20  20     20     20    20   

                             Pos. emb.  Col. emb.  
Model         MoreCapacity                         
ColumnTabBert Field Transf.         20         20  
              Final Transf.         20         20  
FTTransformer Field Transf.         20         20  
Fieldy        Field Transf.         20         20  
              Final Transf.         20         20  
RowTabBert    Field Transf.         20         20  
              Final Transf.         20         20  
Tabbie        Field Transf.         20         20

In [6]:
df_runs_kdd = df_kdd.groupby(["Model", "MoreCapacity", "Pos. emb.", "Col. emb."]).agg(
    n_runs=("seed", "count"),
    AP=("AP", "mean"),
    std_AP=("AP", np.std),
    F1=("F1", "mean"),
    std_F1=("F1", np.std),
).reset_index()
display_df_kdd = df_runs_kdd[["Model", "MoreCapacity", "Pos. emb.", "Col. emb.", "AP", "std_AP"]].sort_values(by=["Model", "MoreCapacity", "Pos. emb.", "Col. emb."])

display_df_kdd.style.format({
    "AP": "{:.2f}",
    "std_AP": "{:.3f}",
})

,Model,MoreCapacity,Pos. emb.,Col. emb.,AP,std_AP
0,ColumnTabBert,Field Transf.,No,No,0.44,0.047
1,ColumnTabBert,Field Transf.,No,Yes,0.43,0.036
2,ColumnTabBert,Field Transf.,Yes,No,0.40,0.077
3,ColumnTabBert,Field Transf.,Yes,Yes,0.42,0.033
4,ColumnTabBert,Final Transf.,No,No,0.37,0.026
5,ColumnTabBert,Final Transf.,No,Yes,0.38,0.021
6,ColumnTabBert,Final Transf.,Yes,No,0.37,0.048
7,ColumnTabBert,Final Transf.,Yes,Yes,0.36,0.040
8,FTTransformer,Field Transf.,No,No,0.43,0.084
9,FTTransformer,Field Transf.,No,Yes,0.42,0.075


In [7]:
df_runs_prsa = df_prsa.groupby(["Model", "MoreCapacity", "# weights", "Pos. emb.", "Col. emb."]).agg(
    n_runs=("seed", "count"),
    RMSE=("RMSE", "mean"),
    std_RMSE=("RMSE", np.std),
    MAPE=("MAPE", "mean"),
    std_MAPE=("MAPE", np.std),
).reset_index()
display_df_prsa = df_runs_prsa[["Model", "MoreCapacity", "# weights", "Pos. emb.", "Col. emb.", "RMSE", "std_RMSE"]].sort_values(by=["Model", "MoreCapacity", "Pos. emb.", "Col. emb."])

display_df_prsa.style.format({
    "# weights": "{:,.0f}", 
    "RMSE": "{:.2f}",
    "std_RMSE": "{:.3f}",
})

,Model,MoreCapacity,# weights,Pos. emb.,Col. emb.,RMSE,std_RMSE
0,ColumnTabBert,Field Transf.,"106,565,590",No,No,27.10,0.323
1,ColumnTabBert,Field Transf.,"106,565,590",No,Yes,27.08,0.321
2,ColumnTabBert,Field Transf.,"106,574,390",Yes,No,26.46,0.320
3,ColumnTabBert,Field Transf.,"106,574,390",Yes,Yes,26.48,0.282
4,ColumnTabBert,Final Transf.,"102,296,534",No,No,27.85,0.346
5,ColumnTabBert,Final Transf.,"102,296,534",No,Yes,27.88,0.300
6,ColumnTabBert,Final Transf.,"102,305,334",Yes,No,27.19,0.275
7,ColumnTabBert,Final Transf.,"102,305,334",Yes,Yes,27.23,0.215
8,FTTransformer,Field Transf.,"107,602,678",No,No,28.28,0.275
10,FTTransformer,Field Transf.,"107,616,278",No,Yes,28.04,0.217


In [8]:
best_df_prsa_latex = display_df_prsa.groupby(["Model"]).agg(
        RMSE=("RMSE", "min"),
).reset_index().merge(
    display_df_prsa[["Model", "RMSE", "std_RMSE"]], 
    how="left", 
    on=["Model", "RMSE"]
)

best_df_kdd_latex = display_df_kdd.groupby(["Model"]).agg(
        AP=("AP", "max"),
).reset_index().merge(
    display_df_kdd[["Model", "MoreCapacity", "AP", "std_AP",]], 
    how="left", 
    on=["Model", "AP"]
)

best_df_latex = best_df_prsa_latex[["Model", "RMSE", "std_RMSE"]].merge(
    best_df_kdd_latex[["Model", "AP", "std_AP"]],
    how="left",
    on="Model",
).drop_duplicates()

best_df_latex["RMSE"] = best_df_latex["RMSE"].round(2).astype(str) + " {\small(" +  best_df_latex["std_RMSE"].round(2).astype(str) +")}"
best_df_latex["AP"] = best_df_latex["AP"].round(2).astype(str) + " {\small(" +  best_df_latex["std_AP"].round(2).astype(str) +")}"

latex_code = best_df_latex[["Model", "RMSE", "AP"]].copy(deep=True)

latex_code["Architecture"] = latex_code["Model"]
latex_code["Architecture"] = latex_code["Architecture"].str.replace("linearbaseline", "Non-DL")
latex_code["Architecture"] = latex_code["Architecture"].str.replace("xgbbaseline", "Non-DL")
latex_code["Architecture"] = latex_code["Architecture"].str.replace("FTTransformer", "Single-stage")
latex_code["Architecture"] = latex_code["Architecture"].str.replace("Tabbie", "Single-stage")

latex_code["Architecture"] = latex_code["Architecture"].str.replace("ColumnTabBert", "Two-stage")
latex_code["Architecture"] = latex_code["Architecture"].str.replace("RowTabBert", "Two-stage")
latex_code["Architecture"] = latex_code["Architecture"].str.replace("Fieldy", "Two-stage")

latex_code = latex_code[["Model", "Architecture", "RMSE", "AP"]].to_latex(index=False, float_format="%.2f")
print(latex_code)

\begin{tabular}{llll}
\toprule
Model & Architecture & RMSE & AP \\
\midrule
ColumnTabBert & Two-stage & 26.46 {\small(0.32)} & 0.44 {\small(0.05)} \\
FTTransformer & Single-stage & 26.54 {\small(0.45)} & 0.44 {\small(0.07)} \\
Fieldy & Two-stage & 20.13 {\small(0.34)} & 0.48 {\small(0.06)} \\
RowTabBert & Two-stage & 21.05 {\small(0.22)} & 0.46 {\small(0.06)} \\
Tabbie & Single-stage & 22.37 {\small(0.31)} & 0.39 {\small(0.05)} \\
\bottomrule
\end{tabular}



In [9]:
display_df_prsa = display_df_prsa[~display_df_prsa["Model"].str.contains("baseline")]
display_df_kdd = display_df_kdd[~display_df_kdd["Model"].str.contains("baseline")]
final_df = display_df_prsa[["Model", "MoreCapacity", "Pos. emb.", "Col. emb.", "RMSE", "std_RMSE"]].merge(
    display_df_kdd[["Model", "MoreCapacity", "Pos. emb.", "Col. emb.", "AP", "std_AP"]],
    how="left",
)

final_df

,Model,MoreCapacity,Pos. emb.,Col. emb.,RMSE,std_RMSE,AP,std_AP
0,ColumnTabBert,Field Transf.,No,No,27.102,0.322676,0.438,0.046583
1,ColumnTabBert,Field Transf.,No,Yes,27.076,0.321450,0.428,0.035637
2,ColumnTabBert,Field Transf.,Yes,No,26.464,0.320047,0.402,0.076942
3,ColumnTabBert,Field Transf.,Yes,Yes,26.482,0.282082,0.418,0.033466
4,ColumnTabBert,Final Transf.,No,No,27.848,0.345861,0.374,0.026077
5,ColumnTabBert,Final Transf.,No,Yes,27.880,0.300250,0.384,0.020736
6,ColumnTabBert,Final Transf.,Yes,No,27.192,0.274809,0.370,0.047958
7,ColumnTabBert,Final Transf.,Yes,Yes,27.226,0.215244,0.364,0.040373
8,FTTransformer,Field Transf.,No,No,28.278,0.274809,0.430,0.083964
9,FTTransformer,Field Transf.,No,Yes,28.040,0.216679,0.418,0.075299


In [10]:
final_df

,Model,MoreCapacity,Pos. emb.,Col. emb.,RMSE,std_RMSE,AP,std_AP
0,ColumnTabBert,Field Transf.,No,No,27.102,0.322676,0.438,0.046583
1,ColumnTabBert,Field Transf.,No,Yes,27.076,0.321450,0.428,0.035637
2,ColumnTabBert,Field Transf.,Yes,No,26.464,0.320047,0.402,0.076942
3,ColumnTabBert,Field Transf.,Yes,Yes,26.482,0.282082,0.418,0.033466
4,ColumnTabBert,Final Transf.,No,No,27.848,0.345861,0.374,0.026077
5,ColumnTabBert,Final Transf.,No,Yes,27.880,0.300250,0.384,0.020736
6,ColumnTabBert,Final Transf.,Yes,No,27.192,0.274809,0.370,0.047958
7,ColumnTabBert,Final Transf.,Yes,Yes,27.226,0.215244,0.364,0.040373
8,FTTransformer,Field Transf.,No,No,28.278,0.274809,0.430,0.083964
9,FTTransformer,Field Transf.,No,Yes,28.040,0.216679,0.418,0.075299


In [11]:
final_df["Pos. emb."] = final_df["Pos. emb."].str.replace("Yes", "\checkmark")
final_df["Pos. emb."] = final_df["Pos. emb."].str.replace("No", "")
final_df["Col. emb."] = final_df["Col. emb."].str.replace("Yes", "\checkmark")
final_df["Col. emb."] = final_df["Col. emb."].str.replace("No", "")

final_df["RMSE"] = final_df["RMSE"].round(2).astype(str) + " {\small(" +  final_df["std_RMSE"].round(2).astype(str) +")}"
final_df["AP"] = final_df["AP"].round(2).astype(str) + " {\small(" +  final_df["std_AP"].round(2).astype(str) +")}"

final_df = final_df[["Model","MoreCapacity", "Col. emb.", "Pos. emb.", "RMSE", "AP"]]
final_df = final_df.rename(columns={"RMSE": "Pollution [RMSE $\downarrow$]"})
final_df = final_df.rename(columns={"AP": "Loan [AP$\\uparrow$]"})

final_df = final_df.sort_values(by=["Model", "MoreCapacity"], ascending=[True, True])
final_df.loc[final_df['Model'].duplicated(), 'Model'] = '-'
final_df

,Model,MoreCapacity,Col. emb.,Pos. emb.,Pollution [RMSE $\downarrow$],Loan [AP$\uparrow$]
0,ColumnTabBert,Field Transf.,,,27.1 {\small(0.32)},0.44 {\small(0.05)}
1,-,Field Transf.,\checkmark,,27.08 {\small(0.32)},0.43 {\small(0.04)}
2,-,Field Transf.,,\checkmark,26.46 {\small(0.32)},0.4 {\small(0.08)}
3,-,Field Transf.,\checkmark,\checkmark,26.48 {\small(0.28)},0.42 {\small(0.03)}
4,-,Final Transf.,,,27.85 {\small(0.35)},0.37 {\small(0.03)}
5,-,Final Transf.,\checkmark,,27.88 {\small(0.3)},0.38 {\small(0.02)}
6,-,Final Transf.,,\checkmark,27.19 {\small(0.27)},0.37 {\small(0.05)}
7,-,Final Transf.,\checkmark,\checkmark,27.23 {\small(0.22)},0.36 {\small(0.04)}
8,FTTransformer,Field Transf.,,,28.28 {\small(0.27)},0.43 {\small(0.08)}
9,-,Field Transf.,\checkmark,,28.04 {\small(0.22)},0.42 {\small(0.08)}


In [12]:
latex_code_ablation = final_df.to_latex(index=False, float_format="%.2f")
print(latex_code_ablation)

\begin{tabular}{llllll}
\toprule
Model & MoreCapacity & Col. emb. & Pos. emb. & Pollution [RMSE $\downarrow$] & Loan [AP$\uparrow$] \\
\midrule
ColumnTabBert & Field Transf. &  &  & 27.1 {\small(0.32)} & 0.44 {\small(0.05)} \\
- & Field Transf. & \checkmark &  & 27.08 {\small(0.32)} & 0.43 {\small(0.04)} \\
- & Field Transf. &  & \checkmark & 26.46 {\small(0.32)} & 0.4 {\small(0.08)} \\
- & Field Transf. & \checkmark & \checkmark & 26.48 {\small(0.28)} & 0.42 {\small(0.03)} \\
- & Final Transf. &  &  & 27.85 {\small(0.35)} & 0.37 {\small(0.03)} \\
- & Final Transf. & \checkmark &  & 27.88 {\small(0.3)} & 0.38 {\small(0.02)} \\
- & Final Transf. &  & \checkmark & 27.19 {\small(0.27)} & 0.37 {\small(0.05)} \\
- & Final Transf. & \checkmark & \checkmark & 27.23 {\small(0.22)} & 0.36 {\small(0.04)} \\
FTTransformer & Field Transf. &  &  & 28.28 {\small(0.27)} & 0.43 {\small(0.08)} \\
- & Field Transf. & \checkmark &  & 28.04 {\small(0.22)} & 0.42 {\small(0.08)} \\
- & Field Transf. &  & \c